# face2rec2.py

In [96]:
import os
import sys

#curr_path = os.path.abspath(os.path.dirname(__file__))
#sys.path.append(os.path.join(curr_path, "../python"))
import mxnet as mx
from skimage import transform as trans


import random
import argparse
import cv2
import time, json
import numpy as np
import traceback
#from builtins import range
from easydict import EasyDict as edict
# sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'common'))
# import face_preprocess
# import face_image

try:
    import multiprocessing
except ImportError:
    multiprocessing = None

### face_process.py

In [97]:
def parse_lst_line(line):
  vec = line.strip().split("\t")
  assert len(vec)>=3
  aligned = int(vec[0])
  image_path = vec[1]
  label = int(vec[2])
  bbox = None
  landmark = None
  #print(vec)
  if len(vec)>3:
    bbox = np.zeros( (4,), dtype=np.int32)
    for i in xrange(3,7):
      bbox[i-3] = int(vec[i])
    landmark = None
    if len(vec)>7:
      _l = []
      for i in xrange(7,17):
        _l.append(float(vec[i]))
      landmark = np.array(_l).reshape( (2,5) ).T
  #print(aligned)
  return image_path, label, bbox, landmark, aligned

In [ ]:
def read_image(img_path, **kwargs):
  mode = kwargs.get('mode', 'rgb')
  layout = kwargs.get('layout', 'HWC')
  if mode=='gray':
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_GRAYSCALE)
  else:
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_COLOR)
    if mode=='rgb':
      #print('to rgb')
      img = img[...,::-1]
    if layout=='CHW':
      img = np.transpose(img, (2,0,1))
  return img

In [8]:
def preprocess(img, bbox=None, landmark=None, **kwargs):
  if isinstance(img, str):
    img = read_image(img, **kwargs)
  M = None
  image_size = []
  str_image_size = kwargs.get('image_size', '')
  if len(str_image_size)>0:
    image_size = [int(x) for x in str_image_size.split(',')]
    if len(image_size)==1:
      image_size = [image_size[0], image_size[0]]
    assert len(image_size)==2
    assert image_size[0]==112
    assert image_size[0]==112 or image_size[1]==96
  if landmark is not None:
    assert len(image_size)==2
    src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==112:
      src[:,0] += 8.0
    dst = landmark.astype(np.float32)

    tform = trans.SimilarityTransform()
    tform.estimate(dst, src)
    M = tform.params[0:2,:]
    #M = cv2.estimateRigidTransform( dst.reshape(1,5,2), src.reshape(1,5,2), False)

  if M is None:
    if bbox is None: #use center crop
      det = np.zeros(4, dtype=np.int32)
      det[0] = int(img.shape[1]*0.0625)
      det[1] = int(img.shape[0]*0.0625)
      det[2] = img.shape[1] - det[0]
      det[3] = img.shape[0] - det[1]
    else:
      det = bbox
    margin = kwargs.get('margin', 44)
    bb = np.zeros(4, dtype=np.int32)
    bb[0] = np.maximum(det[0]-margin/2, 0)
    bb[1] = np.maximum(det[1]-margin/2, 0)
    bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
    bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
    ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
    if len(image_size)>0:
      ret = cv2.resize(ret, (image_size[1], image_size[0]))
    return ret 
  else: #do align using landmark
    assert len(image_size)==2

    #src = src[0:3,:]
    #dst = dst[0:3,:]


    #print(src.shape, dst.shape)
    #print(src)
    #print(dst)
    #print(M)
    warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)

    #tform3 = trans.ProjectiveTransform()
    #tform3.estimate(src, dst)
    #warped = trans.warp(img, tform3, output_shape=_shape)
    return warped

### face_image.py

In [ ]:
def load_property(data_dir):
  prop = edict()
  for line in open(os.path.join(data_dir, 'property')):
    vec = line.strip().split(',')
    assert len(vec)==3
    prop.num_classes = int(vec[0])
    prop.image_size = [int(vec[1]), int(vec[2])]
  return prop

### face2rec2.py

In [98]:
def read_list(path_in):
    with open(path_in) as fin:
        identities = []
        last = [-1, -1]
        _id = 1
        while True:
            line = fin.readline()
            if not line:
                break
            item = edict()
            
            # flog = 0
            item.flag = 0
            item.image_path, label, item.bbox, item.landmark, item.aligned = parse_lst_line(line)
            if not item.aligned and item.landmark is None:
              #print('ignore line', line)
              continue
            item.id = _id
            item.label = [label, item.aligned]
            yield item
            if label!=last[0]:
              if last[1]>=0:
                identities.append( (last[1], _id) )
              last[0] = label
              last[1] = _id
            _id+=1
        identities.append( (last[1], _id) )
        item = edict()
        
        # flag == 2
        item.flag = 2
        item.id = 0
        item.label = [float(_id), float(_id+len(identities))]
        yield item
        for identity in identities:
          item = edict()
          item.flag = 2
          item.id = _id
          _id+=1
          item.label = [float(identity[0]), float(identity[1])]
          yield item

In [118]:
def image_encode(args, i, item, q_out):
#     print('flag:',item.flag)
    oitem = [item.id]
    
    # 개별 이미지에 대한 정보
    if item.flag==0:
        fullpath = item.image_path
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
        print('header:',header)
        
        # align이 되어있을 때(112 * 112)
        if item.aligned:
            with open(fullpath, 'rb') as fin:
                img = fin.read()
            s = mx.recordio.pack(header, img)
#             print('s',s)
            q_out.put((i, s, oitem))
            
        # align이 되어있지 않을 때
        else:
            img = cv2.imread(fullpath, args.color)
            assert item.landmark is not None
            img = preprocess(img, bbox = item.bbox, landmark=item.landmark, image_size='%d,%d'%(args.image_h, args.image_w))
            s = mx.recordio.pack_img(header, img, quality=args.quality, img_fmt=args.encoding)
            
            print('s', s)
            q_out.put((i, s, oitem))
            
    # dir에 대한 메타 정보
    else: 
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
        print('header:', header)
        #print('write', item.flag, item.id, item.label)
        s = mx.recordio.pack(header, b'')
        print('s', s)
        q_out.put((i, s, oitem))

# main()

In [119]:
args = edict({
    'image_h': 112,
    'image_w': 112,
    'prefix': '../data/small_vgg',
    'root': '../data/small_vgg/small_vgg_112x112',
    'encoding':'.jpg',
    'quality': 95,
})

In [120]:
working_dir = args.prefix
prop = load_property(working_dir)
image_size = prop.image_size
print('image_size', image_size)

image_size [112, 112]


In [121]:
files = [os.path.join(working_dir, fname).replace('\\','/') for fname in os.listdir(working_dir)
        if os.path.isfile(os.path.join(working_dir, fname))]
print(files)

['../data/small_vgg/property', '../data/small_vgg/small_vgg_112x112.idx', '../data/small_vgg/small_vgg_112x112.lst', '../data/small_vgg/small_vgg_112x112.rec']


In [122]:
count = 0
for fname in files:
    # .lst file
    if fname.startswith(args.prefix) and fname.endswith('.lst'):
        print('Creating .rec file from', fname, 'in', working_dir)
        count += 1
        image_list = read_list(fname)

        # -- write_record -- #
        try:
            import Queue as queue
        except ImportError:
            import queue
        q_out = queue.Queue()
        fname = os.path.basename(fname)
        fname_rec = os.path.splitext(fname)[0] + '.rec'
        fname_idx = os.path.splitext(fname)[0] + '.idx'
        record = mx.recordio.MXIndexedRecordIO(os.path.join(working_dir, fname_idx),
                                               os.path.join(working_dir, fname_rec), 'w')
        cnt = 0
        pre_time = time.time()
        for i, item in enumerate(image_list):
            image_encode(args, i, item, q_out)
            if q_out.empty():
                continue
            _, s, item = q_out.get()
            #header, _ = mx.recordio.unpack(s)
            #print('write header label', header.label)
            print(item[0])
            record.write_idx(item[0], s)
            if cnt % 1000 == 0:
                cur_time = time.time()
                print('time:', cur_time - pre_time, ' count:', cnt)
                pre_time = cur_time
            cnt += 1

Creating .rec file from ../data/small_vgg/small_vgg_112x112.lst in ../data/small_vgg
header: HEADER(flag=0, label=[0, 1], id=1, id2=0)
1
time: 0.0009999275207519531  count: 0
header: HEADER(flag=0, label=[0, 1], id=2, id2=0)
2
header: HEADER(flag=0, label=[0, 1], id=3, id2=0)
3
header: HEADER(flag=0, label=[0, 1], id=4, id2=0)
4
header: HEADER(flag=0, label=[0, 1], id=5, id2=0)
5
header: HEADER(flag=0, label=[0, 1], id=6, id2=0)
6
header: HEADER(flag=0, label=[0, 1], id=7, id2=0)
7
header: HEADER(flag=0, label=[0, 1], id=8, id2=0)
8
header: HEADER(flag=0, label=[0, 1], id=9, id2=0)
9
header: HEADER(flag=0, label=[0, 1], id=10, id2=0)
10
header: HEADER(flag=0, label=[0, 1], id=11, id2=0)
11
header: HEADER(flag=0, label=[0, 1], id=12, id2=0)
12
header: HEADER(flag=0, label=[0, 1], id=13, id2=0)
13
header: HEADER(flag=0, label=[0, 1], id=14, id2=0)
14
header: HEADER(flag=0, label=[0, 1], id=15, id2=0)
15
header: HEADER(flag=0, label=[0, 1], id=16, id2=0)
16
header: HEADER(flag=0, label=[0

163
header: HEADER(flag=0, label=[0, 1], id=164, id2=0)
164
header: HEADER(flag=0, label=[0, 1], id=165, id2=0)
165
header: HEADER(flag=0, label=[0, 1], id=166, id2=0)
166
header: HEADER(flag=0, label=[0, 1], id=167, id2=0)
167
header: HEADER(flag=0, label=[0, 1], id=168, id2=0)
168
header: HEADER(flag=0, label=[0, 1], id=169, id2=0)
169
header: HEADER(flag=0, label=[0, 1], id=170, id2=0)
170
header: HEADER(flag=0, label=[0, 1], id=171, id2=0)
171
header: HEADER(flag=0, label=[0, 1], id=172, id2=0)
172
header: HEADER(flag=0, label=[0, 1], id=173, id2=0)
173
header: HEADER(flag=0, label=[0, 1], id=174, id2=0)
174
header: HEADER(flag=0, label=[0, 1], id=175, id2=0)
175
header: HEADER(flag=0, label=[0, 1], id=176, id2=0)
176
header: HEADER(flag=0, label=[0, 1], id=177, id2=0)
177
header: HEADER(flag=0, label=[0, 1], id=178, id2=0)
178
header: HEADER(flag=0, label=[0, 1], id=179, id2=0)
179
header: HEADER(flag=0, label=[0, 1], id=180, id2=0)
180
header: HEADER(flag=0, label=[0, 1], id=181,

332
header: HEADER(flag=0, label=[0, 1], id=333, id2=0)
333
header: HEADER(flag=0, label=[0, 1], id=334, id2=0)
334
header: HEADER(flag=0, label=[0, 1], id=335, id2=0)
335
header: HEADER(flag=0, label=[0, 1], id=336, id2=0)
336
header: HEADER(flag=0, label=[0, 1], id=337, id2=0)
337
header: HEADER(flag=0, label=[0, 1], id=338, id2=0)
338
header: HEADER(flag=0, label=[0, 1], id=339, id2=0)
339
header: HEADER(flag=0, label=[0, 1], id=340, id2=0)
340
header: HEADER(flag=0, label=[0, 1], id=341, id2=0)
341
header: HEADER(flag=0, label=[0, 1], id=342, id2=0)
342
header: HEADER(flag=0, label=[0, 1], id=343, id2=0)
343
header: HEADER(flag=0, label=[0, 1], id=344, id2=0)
344
header: HEADER(flag=0, label=[0, 1], id=345, id2=0)
345
header: HEADER(flag=0, label=[0, 1], id=346, id2=0)
346
header: HEADER(flag=0, label=[0, 1], id=347, id2=0)
347
header: HEADER(flag=0, label=[0, 1], id=348, id2=0)
348
header: HEADER(flag=0, label=[0, 1], id=349, id2=0)
349
header: HEADER(flag=0, label=[0, 1], id=350,

In [110]:
#iterator
image_list.__next__()

{'flag': 0,
 'image_path': '../data/small_vgg/small_vgg_112x112/train/n000001/0001_01.jpg',
 'bbox': None,
 'landmark': None,
 'aligned': 1,
 'id': 1,
 'label': [0, 1]}

# 이미지에 대한 정보
### flag: 0
- 개별 이미지에 대한 정보(모두 295개), 

> 1st dir : 241개
- header: HEADER(flag=2, label=[1.0, 242.0], id=296, id2=0)

> 2nd dir: 27개
- header: HEADER(flag=2, label=[242.0, 269.0], id=297, id2=0)

> 3rd dir: 27개
- header: HEADER(flag=2, label=[269.0, 296.0], id=298, id2=0)



In [10]:
def read_worker(args, q_in, q_out):
    while True:
        deq = q_in.get()
        if deq is None:
            break
        i, item = deq
        image_encode(args, i, item, q_out)

def write_worker(q_out, fname, working_dir):
    pre_time = time.time()
    count = 0
    fname = os.path.basename(fname)
    fname_rec = os.path.splitext(fname)[0] + '.rec'
    fname_idx = os.path.splitext(fname)[0] + '.idx'
    record = mx.recordio.MXIndexedRecordIO(os.path.join(working_dir, fname_idx),
                                           os.path.join(working_dir, fname_rec), 'w')
    buf = {}
    more = True
    while more:
        deq = q_out.get()
        if deq is not None:
            i, s, item = deq
            buf[i] = (s, item)
        else:
            more = False
        while count in buf:
            s, item = buf[count]
            del buf[count]
            if s is not None:
                #print('write idx', item[0])
                record.write_idx(item[0], s)

            if count % 1000 == 0:
                cur_time = time.time()
                print('time:', cur_time - pre_time, ' count:', count)
                pre_time = cur_time
            count += 1

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Create an image list or \
        make a record database by reading from an image list')
    parser.add_argument('prefix', help='prefix of input/output lst and rec files.')
    #parser.add_argument('root', help='path to folder containing images.')

    cgroup = parser.add_argument_group('Options for creating image lists')
    cgroup.add_argument('--list', type=bool, default=False,
                        help='If this is set im2rec will create image list(s) by traversing root folder\
        and output to <prefix>.lst.\
        Otherwise im2rec will read <prefix>.lst and create a database at <prefix>.rec')
    cgroup.add_argument('--exts', nargs='+', default=['.jpeg', '.jpg'],
                        help='list of acceptable image extensions.')
    cgroup.add_argument('--chunks', type=int, default=1, help='number of chunks.')
    cgroup.add_argument('--train-ratio', type=float, default=1.0,
                        help='Ratio of images to use for training.')
    cgroup.add_argument('--test-ratio', type=float, default=0,
                        help='Ratio of images to use for testing.')
    cgroup.add_argument('--recursive', type=bool, default=False,
                        help='If true recursively walk through subdirs and assign an unique label\
        to images in each folder. Otherwise only include images in the root folder\
        and give them label 0.')
    cgroup.add_argument('--shuffle', type=bool, default=True, help='If this is set as True, \
        im2rec will randomize the image order in <prefix>.lst')

    rgroup = parser.add_argument_group('Options for creating database')
    rgroup.add_argument('--quality', type=int, default=95,
                        help='JPEG quality for encoding, 1-100; or PNG compression for encoding, 1-9')
    rgroup.add_argument('--num-thread', type=int, default=1,
                        help='number of thread to use for encoding. order of images will be different\
        from the input list if >1. the input list will be modified to match the\
        resulting order.')
    rgroup.add_argument('--color', type=int, default=1, choices=[-1, 0, 1],
                        help='specify the color mode of the loaded image.\
        1: Loads a color image. Any transparency of image will be neglected. It is the default flag.\
        0: Loads image in grayscale mode.\
        -1:Loads image as such including alpha channel.')
    rgroup.add_argument('--encoding', type=str, default='.jpg', choices=['.jpg', '.png'],
                        help='specify the encoding of the images.')
    rgroup.add_argument('--pack-label', type=bool, default=False,
        help='Whether to also pack multi dimensional label in the record file')
    args = parser.parse_args()
    args.prefix = os.path.abspath(args.prefix)
    #args.root = os.path.abspath(args.root)
    return args